# 1. Получение данных

1.1. import roboflow

In [1]:
%pip install -q roboflow
from roboflow import Roboflow

rf = Roboflow(model_format='yolov5', notebook='ultralytics')

     |████████████████████████████████| 41 kB 728 kB/s 
     |████████████████████████████████| 145 kB 65.3 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
     |████████████████████████████████| 67 kB 7.6 MB/s 
     |████████████████████████████████| 138 kB 75.0 MB/s 
     |████████████████████████████████| 178 kB 56.7 MB/s 
     |████████████████████████████████| 62 kB 713 kB/s 
upload and label your dataset, and get an API KEY here: https://app.roboflow.com/?model=yolov5&ref=ultralytics


1.2 Загрузка датасета

In [2]:
# https://universe.roboflow.com/mohamed-traore-2ekkp/taco-trash-annotations-in-context
# не нашел как форкнуть к себе из universe
from roboflow import Roboflow
rf = Roboflow(api_key="jFwPkcSyDZ9TxcgPTyz6")
project = rf.workspace("mohamed-traore-2ekkp").project("taco-trash-annotations-in-context")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to TACO:-Trash-Annotations-in-Context-Dataset-2 in yolov5pytorch:: 100%|██████████| 7196/7196 [00:03<00:00, 2358.87it/s]


# 2. Распознавание объектов

2.1 Клонирование репо YOLOv5

In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt
%mv ../TACO:-Trash-Annotations-in-Context-Dataset-2 .

Cloning into 'yolov5'...
remote: Enumerating objects: 15093, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 15093 (delta 0), reused 3 (delta 0), pack-reused 15088
Receiving objects: 100% (15093/15093), 13.63 MiB | 15.98 MiB/s, done.
Resolving deltas: 100% (10436/10436), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 22.2 MB/s 


2.2 Обучение модели

In [4]:
!python train.py --img 416 --batch 16 --epochs 100 --data TACO:-Trash-Annotations-in-Context-Dataset-2/data.yaml --weights yolov5s.pt --cache 

train: weights=yolov5s.pt, cfg=, data=TACO:-Trash-Annotations-in-Context-Dataset-2/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-249-gff6e6e3 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gam

2.3 Детектирование объектов на изображениях из тестовой выборки

In [23]:
!python detect.py --weights runs/train/exp/weights/best.pt --img 416 --save-txt --conf 0.1 --source TACO:-Trash-Annotations-in-Context-Dataset-2/test/images

detect: weights=['runs/train/exp/weights/best.pt'], source=TACO:-Trash-Annotations-in-Context-Dataset-2/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-249-gff6e6e3 Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 157 layers, 7169248 parameters, 0 gradients, 16.3 GFLOPs
image 1/150 /content/yolov5/TACO:-Trash-Annotations-in-Context-Dataset-2/test/images/000000_JPG.rf.3144ee029a987258a2aaaf0ebf2b01c8.jpg: 416x416 1 Drink carton, 1 Normal paper, 8.4ms
image 2/150 /content/yolov5/TACO:-Trash-Annotations-in-Context-Dataset-2/test/images/000000_JPG.rf.fadf92df9bf682513fc56e64a10ff

# 3. Форматирование результатов распознавания

3.1 Соответствие между лейблами и названием

In [24]:
import re

classes = dict()
with open("TACO:-Trash-Annotations-in-Context-Dataset-2/data.yaml") as file:
  for index, line in enumerate(file):
    if  "- " in line:
      line = re.split('- ', line)
      name = line[1][:-1]
      classes[index - 1] = name

classes

{0: 'Aerosol',
 1: 'Aluminium blister pack',
 2: 'Aluminium foil',
 3: 'Battery',
 4: 'Broken glass',
 5: 'Carded blister pack',
 6: 'Cigarette',
 7: 'Clear plastic bottle',
 8: 'Corrugated carton',
 9: 'Crisp packet',
 10: 'Disposable food container',
 11: 'Disposable plastic cup',
 12: 'Drink can',
 13: 'Drink carton',
 14: 'Egg carton',
 15: 'Foam cup',
 16: 'Foam food container',
 17: 'Food Can',
 18: 'Food waste',
 19: 'Garbage bag',
 20: 'Glass bottle',
 21: 'Glass cup',
 22: 'Glass jar',
 23: 'Magazine paper',
 24: 'Meal carton',
 25: 'Metal bottle cap',
 26: 'Metal lid',
 27: 'Normal paper',
 28: 'Other carton',
 29: 'Other plastic bottle',
 30: 'Other plastic container',
 31: 'Other plastic cup',
 32: 'Other plastic wrapper',
 33: 'Other plastic',
 34: 'Paper bag',
 35: 'Paper cup',
 36: 'Paper straw',
 37: 'Pizza box',
 38: 'Plastic bottle cap',
 39: 'Plastic film',
 40: 'Plastic glooves',
 41: 'Plastic lid',
 42: 'Plastic straw',
 43: 'Plastic utensils',
 44: 'Polypropylene 

3.2 Формирование датафрейма

In [25]:
import pandas as pd
import os

df = pd.DataFrame(columns=['objects'])

det_n = 4
dir = f"/content/yolov5/runs/detect/exp{det_n}/labels"
for filename in os.listdir(dir):
  with open(os.path.join(dir, filename), 'r') as f:
    text = f.readlines()
    objects = []
    for i in text:
      label = int(i.split(' ')[0])
      objects.append(classes[label])
      index = filename[:-3]+"jpg"
      df.loc[index, 'objects'] = objects

df.head()

,objects
000017_jpg.rf.9529e11095ce388051baf49611ab9e69.jpg,[Glass bottle]
000008_jpg.rf.39105ce5a438b0c8c38f89bfd7db7f71.jpg,[Single-use carrier bag]
000083_JPG.rf.c6cf3f118943d75b17178296822697fe.jpg,[Plastic film]
000020_JPG.rf.d428c5337c54dd6932f273a3c8b2f98b.jpg,"[Other plastic, Other plastic bottle]"
IMG_4865_JPG.rf.74e694a03f8b1d2760e976e34ea83a41.jpg,"[Other carton, Other carton, Plastic film, Cor..."


3.3 Запись датафрейма

In [26]:
df.to_csv('/content/result.csv')

# 4. Извлечение отношений

4.1 One-hot encoding

In [27]:
def encode(image, objects):
  for x in objects:
    data.loc[image, x] = True

data = pd.DataFrame(index = df.index,
                    columns = classes.values())

for index, row in df.iterrows():
  encode(index, row['objects'])
data = data.fillna(False)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102 entries, 000017_jpg.rf.9529e11095ce388051baf49611ab9e69.jpg to 000031_JPG.rf.fe2600f1d33fbfeb0dc6be764bff44c7.jpg
Data columns (total 59 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   Aerosol                    102 non-null    bool 
 1   Aluminium blister pack     102 non-null    bool 
 2   Aluminium foil             102 non-null    bool 
 3   Battery                    102 non-null    bool 
 4   Broken glass               102 non-null    bool 
 5   Carded blister pack        102 non-null    bool 
 6   Cigarette                  102 non-null    bool 
 7   Clear plastic bottle       102 non-null    bool 
 8   Corrugated carton          102 non-null    bool 
 9   Crisp packet               102 non-null    bool 
 10  Disposable food container  102 non-null    bool 
 11  Disposable plastic cup     102 non-null    bool 
 12  Drink can                  102 non-null    

4.2 Itemsets

In [28]:
from mlxtend.frequent_patterns import apriori

min_support = 2/len(data.index)
itemsets = apriori(data,
                   min_support = min_support,
                   max_len = 2,
                   use_colnames = True)

itemsets.head()

,support,itemsets
0,0.029412,(Aerosol)
1,0.078431,(Cigarette)
2,0.215686,(Clear plastic bottle)
3,0.029412,(Corrugated carton)
4,0.039216,(Crisp packet)


4.3 Поиск ассоциативных правил

In [29]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(itemsets, min_threshold = min_support)
rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x)[0])
rules['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])

rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Aerosol,Other plastic container,0.029412,0.039216,0.019608,0.666667,17.000000,0.018454,2.882353
1,Other plastic container,Aerosol,0.039216,0.029412,0.019608,0.500000,17.000000,0.018454,1.941176
2,Cigarette,Disposable plastic cup,0.078431,0.049020,0.019608,0.250000,5.100000,0.015763,1.267974
3,Disposable plastic cup,Cigarette,0.049020,0.078431,0.019608,0.400000,5.100000,0.015763,1.535948
4,Cigarette,Drink can,0.078431,0.107843,0.019608,0.250000,2.318182,0.011150,1.189542


4.4 Отбор правил по lift

In [30]:
# выявляем наиболее устойчивые связи между объектами
rels = rules[rules['lift'] > 1]
rels = rels[['antecedents', 'consequents']]

rels.head()

,antecedents,consequents
0,Aerosol,Other plastic container
1,Other plastic container,Aerosol
2,Cigarette,Disposable plastic cup
3,Disposable plastic cup,Cigarette
4,Cigarette,Drink can


# 5. Формирование графа знаний

5.1 Импорт owlready2

In [31]:
!pip install Cython
!pip install owlready2
from owlready2 import *

onto = get_ontology('http://onto.owl')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


5.2 Создание индивидов

In [32]:
import types

with onto:
  onto_cl = types.new_class('Trash', (Thing,))
  for i, cl in enumerate(classes.values()):
    inst = onto_cl(f'trash_{i}')
    inst.label = cl

onto.save('/content/onto.owl')

5.3 Добавление связей

In [33]:
with onto:
  types.new_class('isAssociatedWith', (ObjectProperty, TransitiveProperty))
  for index, rel in rels.iterrows():
    onto_ant = onto.search_one(label = rel['antecedents'])
    onto_con = onto.search_one(label = rel['consequents'])
    onto_ant.isAssociatedWith.append(onto_con)

onto.save('/content/onto.owl')

5.4 Ризонинг для выявления новых связей

In [34]:
with onto:
  sync_reasoner_pellet(infer_property_values=True)

onto.save('/content/onto_reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/antlr-runtime-3.2.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/httpclient-4.2.3.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/log4j-1.2.16.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/usr/local/lib/python3.7/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.7/di

* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_9
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_7
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_42
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_33
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_8
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_0
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_29
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_25
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_28
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_20
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_38
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_34
* Owlready * Adding relation onto.trash_6 isAssociatedWith onto.trash_41
* Owlready * Adding relation onto.trash_6 isAssociatedW

* Owlready2 * Pellet took 1.3330538272857666 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)
